# Multi-View Geometry: Interactive Exploration

This notebook provides an interactive walkthrough of the multi-view geometry pipeline, from camera setup through 3D reconstruction.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from src.feature_matching import (
    build_camera_intrinsics, build_rotation_matrix, build_projection_matrix,
    generate_3d_scene_points, generate_synthetic_correspondences, add_gaussian_noise
)
from src.fundamental_matrix import (
    eight_point_algorithm, compute_analytical_fundamental, sampson_distance
)
from src.epipolar_geometry import (
    compute_epipolar_lines, compute_epipoles_svd,
    compute_epipoles_from_lines, compute_epipoles_from_cameras
)
from src.essential_matrix import compute_essential_matrix, recover_camera_pose
from src.triangulation import linear_triangulation, compute_reprojection_error
from src.visualization import (
    plot_projected_points, plot_epipolar_lines,
    plot_3d_points, plot_reprojection_errors
)

## 1. Camera Setup

Define a stereo camera pair with known intrinsics and relative pose.

In [ ]:
K1 = build_camera_intrinsics(au=100, av=120, u0=128, v0=128)
K2 = build_camera_intrinsics(au=90, av=110, u0=128, v0=128)

R1 = np.eye(3)
t1 = np.zeros(3)
R2 = build_rotation_matrix((0.1, np.pi/4, 0.2))
t2 = np.array([-1000, 190, 230])

P1 = build_projection_matrix(K1, R1, t1)
P2 = build_projection_matrix(K2, R2, t2)

print('Camera 1 intrinsics:\n', K1)
print('\nCamera 2 intrinsics:\n', K2)
print('\nP1 shape:', P1.shape)
print('P2 shape:', P2.shape)

## 2. Generate Point Correspondences

In [ ]:
points_3d = generate_3d_scene_points()
pts1, pts2 = generate_synthetic_correspondences(points_3d, P1, P2)

print(f'Generated {pts1.shape[1]} correspondences')
plot_projected_points(pts1, pts2, (256, 256))
plt.show()

## 3. Fundamental Matrix Estimation

In [ ]:
F_analytical = compute_analytical_fundamental(K1, K2, R2, t2)
F_computed = eight_point_algorithm(pts1, pts2, enforce_rank=True, normalize=True)

print('Analytical F:\n', F_analytical)
print('\nComputed F:\n', F_computed)
print(f'\nFrobenius error: {np.linalg.norm(F_analytical - F_computed, "fro"):.6f}')
print(f'Rank of computed F: {np.linalg.matrix_rank(F_computed)}')

## 4. Epipolar Geometry

In [ ]:
ep1, ep2 = compute_epipoles_svd(F_computed)
_, _, coeffs1, coeffs2 = compute_epipolar_lines(pts1, pts2, F_computed)

print(f'Epipole 1: {ep1}')
print(f'Epipole 2: {ep2}')

margins = ((-400, 300), (1, 400))
plot_epipolar_lines(pts1, pts2, coeffs1, coeffs2, ep1, ep2, (256, 256), margins)
plt.show()

## 5. Triangulation and 3D Reconstruction

In [ ]:
points_3d_est = linear_triangulation(P1, P2, pts1, pts2)
errors1, mean1 = compute_reprojection_error(points_3d_est, pts1, P1)
errors2, mean2 = compute_reprojection_error(points_3d_est, pts2, P2)

print(f'Mean reprojection error (Cam 1): {mean1:.6f} px')
print(f'Mean reprojection error (Cam 2): {mean2:.6f} px')

plot_3d_points(points_3d_est)
plt.show()

plot_reprojection_errors(errors1, errors2)
plt.show()